In [ ]:
import importlib
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

import logging
logging.basicConfig(level= logging.INFO)

In [2]:
cust = pd.read_csv("../dataset/crm/customers.csv.gz")
orde = pd.read_csv("../dataset/crm/orders.csv.gz")
ord_itm = pd.read_csv("../dataset/crm/order_items.csv.gz")
kic = pd.read_csv("../dataset/crm/kitchen_logs.csv.gz")
rev = pd.read_csv("../dataset/crm/customer_reviews.csv.gz")

sup = pd.read_csv("../dataset/erp/suppliers.csv.gz")
ing = pd.read_csv("../dataset/erp/ingredients.csv.gz")
sup_ing = pd.read_csv("../dataset/erp/supplier_ingredients.csv.gz")
res = pd.read_csv("../dataset/erp/restaurants.csv.gz")
inv = pd.read_csv("../dataset/erp/inventory.csv.gz")
menu = pd.read_csv("../dataset/erp/menu_items.csv.gz")
recp = pd.read_csv("../dataset/erp/recipe.csv.gz")
emp = pd.read_csv("../dataset/erp/employees.csv.gz")
del_part = pd.read_csv("../dataset/erp/delivery_partners.csv.gz")

In [95]:
a = pd.DataFrame({
    'id':['R001','02','R000','R#04','R0 5','0000',np.nan,'nan'],
    'name':['name_1','name_','name_00003',np.nan,'name_05','name6','nme_7',np.nan]
})

In [ ]:
id = a['id'].str.replace(r'\D','',regex=True).replace({'':np.nan})
id1 = pd.to_numeric(id,errors='coerce').replace(0,np.nan).astype('Int16')
id2 = id1.fillna(a['name'].replace(r'\D','',regex=True).replace({'':np.nan}))
id3 = ('R'+id2.astype(str).str.zfill(3)).where(id2.notnull(), np.nan)


name = id3.replace(r'\D','',regex=True).astype('Int16')
('Restaurant_'+name.astype(str)).where(name.notna(),np.nan)

0    Restaurant_1
1    Restaurant_2
2    Restaurant_3
3    Restaurant_4
4    Restaurant_5
5    Restaurant_6
6    Restaurant_7
7             NaN
Name: id, dtype: object

In [134]:
id3

0    R001
1    R002
2    R003
3    R004
4    R005
5    R006
6    R007
7     NaN
Name: id, dtype: object

In [129]:
a['id'].str.replace(r'\D','',regex=True)

0     001
1      02
2     000
3      04
4      05
5    0000
6     NaN
7        
Name: id, dtype: object

In [125]:
id.astype('Int16')
pd.to_numeric(id,errors='coerce')

0    1.0
1    2.0
2    0.0
3    4.0
4    5.0
5    0.0
6    NaN
7    NaN
Name: id, dtype: float64

In [ ]:
id = a['id'].str.replace(r'\D','',regex=True).replace({'nan':np.nan,'':np.nan})
id1 = pd.to_numeric(id,errors='coerce').replace(0,np.nan).astype('Int16')
id2 = id1.fillna(a['name'].str.replace(r'\D','',regex=True).replace({'nan':np.nan,'':np.nan}))
id3 = ('R'+id2.astype(str).str.zfill(3)).where(id2.notnull(), np.nan)

0     True
1     True
2     True
3     True
4     True
5     True
6     True
7    False
Name: id, dtype: bool

In [58]:
id =    df['id'].str.replace(r'\D','', regex=True)
id =    id.fillna(df['name'].str.replace(r'\D', '', regex=True).replace({'':np.nan}))
id =    ('I' + id.astype('Int32').astype(str).str.zfill(4)).where(id.notnull() == True, np.nan)

In [59]:
id

0    I0001
1      NaN
2    I0002
3      NaN
Name: id, dtype: object

In [43]:
df

,id,name
0,I0001,Menu_Item_1
1,NaN,Menu_item_
2,I0002,
3,NaN,
